# Indexación de sentencias judiciales en Pinecone

## Objetivo
Cargar las sentencias del archivo Excel en `../Datos/sentencias_pasadas.xlsx`, generar embeddings con **OpenAI** e indexar los vectores en **Pinecone** para búsqueda semántica.

## Requisitos previos
- **OpenAI API Key**: para el modelo de embeddings (`text-embedding-3-small`).
- **Pinecone API Key**: para crear/usar un índice en [Pinecone](https://app.pinecone.io/).

## Configuración de claves (seguridad)
Las claves **no** se escriben en el código. Se leen desde variables de entorno:
1. Crea un archivo `.env` en la carpeta **Resultados** (junto a este notebook).
2. Añade las líneas (sustituye por tus claves reales):
   ```
   OPENAI_API_KEY=sk-...
   PINECONE_API_KEY=...
   ```
3. El archivo `.env` no debe subirse a control de versiones (ya está en `.gitignore` si usas Git).

Puedes copiar `.env.example` a `.env` y rellenar los valores.

---
## 1. Imports y configuración

In [ ]:
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec, Metric

# Cargar variables desde .env (en la carpeta Resultados, junto a este notebook)
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError(
        "Falta OPENAI_API_KEY. Configúrala en el archivo .env (ver celdas de documentación)."
    )
if not PINECONE_API_KEY:
    raise ValueError(
        "Falta PINECONE_API_KEY. Configúrala en el archivo .env (ver celdas de documentación)."
    )

print("Variables de entorno cargadas correctamente.")

---
## 2. Carga y preparación de datos

Se lee el Excel de sentencias. Las columnas pueden tener valores faltantes; se construye un texto único para embedding a partir de **sintesis**, **Resuelve** y **Tema - subtema** (los que existan y tengan contenido). El resto de columnas se conservan como metadatos para filtrado en Pinecone.

In [ ]:
# Ruta al Excel: desde Resultados/ se sube a ../Datos/ (ejecutar el notebook desde la carpeta Resultados)
PATH_EXCEL = Path("../Datos/sentencias_pasadas.xlsx")
if not PATH_EXCEL.exists():
    PATH_EXCEL = Path("Datos/sentencias_pasadas.xlsx")  # por si se ejecuta desde la raíz del proyecto
if not PATH_EXCEL.exists():
    raise FileNotFoundError(f"No se encontró el archivo: {PATH_EXCEL}")

df = pd.read_excel(PATH_EXCEL, engine="openpyxl")

# Eliminar columnas completamente vacías para simplificar
df = df.dropna(axis=1, how="all")

print("Columnas disponibles:", list(df.columns))
print(f"\nRegistros cargados: {len(df)}")
df.head(2)

In [ ]:
# Columnas usadas para construir el texto a embedir (prioridad: sintesis > Resuelve > Tema)
COLUMNAS_TEXTO = ["sintesis", "Resuelve", "Tema - subtema"]

def construir_texto_embedding(fila: pd.Series) -> str:
    """Concatena en un solo texto las columnas con contenido (evita NaN y cadenas vacías)."""
    partes = []
    for col in COLUMNAS_TEXTO:
        if col in fila.index:
            val = fila[col]
            if pd.notna(val) and str(val).strip():
                partes.append(str(val).strip())
    return " \n ".join(partes) if partes else ""

df["texto_embedding"] = df.apply(construir_texto_embedding, axis=1)

# Filtrar filas sin texto para embedir
df_valid = df[df["texto_embedding"].str.len() > 0].copy()
df_valid = df_valid.reset_index(drop=True)

print(f"Registros con texto para embedding: {len(df_valid)} de {len(df)}")

---
## 3. Embeddings con OpenAI

Se usa el modelo `text-embedding-3-small` (dimensión 1536). Las peticiones se envían por lotes para no superar límites de la API.

In [ ]:
EMBEDDING_MODEL = "text-embedding-3-small"
DIMENSION = 1536
BATCH_SIZE = 100

client_openai = OpenAI(api_key=OPENAI_API_KEY)

def obtener_embeddings(textos: list[str]) -> list[list[float]]:
    """Obtiene embeddings por lotes usando la API de OpenAI."""
    resultados = []
    for i in range(0, len(textos), BATCH_SIZE):
        lote = textos[i : i + BATCH_SIZE]
        # La API no acepta cadenas vacías
        lote = [t if t and t.strip() else " " for t in lote]
        resp = client_openai.embeddings.create(model=EMBEDDING_MODEL, input=lote)
        orden = sorted(resp.data, key=lambda x: x.index)
        resultados.extend([e.embedding for e in orden])
    return resultados

textos = df_valid["texto_embedding"].astype(str).tolist()
print(f"Generando embeddings para {len(textos)} textos (modelo: {EMBEDDING_MODEL})...")
embeddings = obtener_embeddings(textos)
print(f"Obtenidos {len(embeddings)} vectores de dimensión {len(embeddings[0]) if embeddings else 0}.")

---
## 4. Índice Pinecone

Se crea un índice serverless (si no existe) con dimensión 1536 y métrica coseno. Luego se hace **upsert** de los vectores con metadatos (tipo, fecha, tema, etc.) para poder filtrar en búsquedas.

In [ ]:
INDEX_NAME = "sentencias-judiciales"
REGION = "us-east-1"

pc = Pinecone(api_key=PINECONE_API_KEY)

if INDEX_NAME not in [idx.name for idx in pc.list_indexes()]:
    pc.create_index(
        name=INDEX_NAME,
        dimension=DIMENSION,
        metric=Metric.COSINE,
        spec=ServerlessSpec(cloud="aws", region=REGION),
    )
    print(f"Índice '{INDEX_NAME}' creado.")
else:
    print(f"Índice '{INDEX_NAME}' ya existe.")

index = pc.Index(INDEX_NAME)

In [ ]:
# Preparar metadatos para Pinecone (solo tipos JSON-serializables: str, int, float, bool)
import numpy as np

def _a_nativo(val):
    """Convierte numpy/pandas a tipos nativos de Python para que Pinecone pueda serializar."""
    if pd.isna(val):
        return None
    if isinstance(val, (np.integer, np.int64, np.int32)):
        return int(val)
    if isinstance(val, (np.floating, np.float64, np.float32)):
        return float(val)
    if isinstance(val, np.bool_):
        return bool(val)
    if isinstance(val, str):
        return val[:40_000] if len(val) > 40_000 else val  # Límite Pinecone por valor
    if isinstance(val, (int, float, bool)):
        return val
    return str(val)

def fila_a_metadata(fila: pd.Series) -> dict:
    meta = {}
    for col in df_valid.columns:
        if col == "texto_embedding":
            continue
        val = fila[col]
        if pd.isna(val):
            continue
        v = _a_nativo(val)
        if v is not None:
            meta[col] = v
    return meta

UPSERT_BATCH = 100
ids = [f"sentencia_{i}" for i in range(len(df_valid))]

for i in range(0, len(ids), UPSERT_BATCH):
    batch_ids = ids[i : i + UPSERT_BATCH]
    batch_vectors = embeddings[i : i + UPSERT_BATCH]
    batch_meta = [fila_a_metadata(df_valid.iloc[j]) for j in range(i, min(i + UPSERT_BATCH, len(df_valid)))]
    vectors = [
        {"id": id_, "values": vec, "metadata": meta}
        for id_, vec, meta in zip(batch_ids, batch_vectors, batch_meta)
    ]
    index.upsert(vectors=vectors)
    print(f"Upsert {i + len(batch_ids)} / {len(ids)}")

print("Indexación en Pinecone completada.")

---
## 5. Comprobación

Consulta de estadísticas del índice y una búsqueda por similitud de ejemplo.

In [ ]:
stats = index.describe_index_stats()
print("Estadísticas del índice:", stats)

In [ ]:
# Ejemplo: buscar sentencias similares a una consulta en lenguaje natural
consulta = "indemnización por despido"
resp_embed = client_openai.embeddings.create(model=EMBEDDING_MODEL, input=[consulta])
vector_consulta = resp_embed.data[0].embedding

resultados = index.query(vector=vector_consulta, top_k=3, include_metadata=True)
print(f"Top 3 resultados para: '{consulta}'")
for m in resultados.matches:
    meta = m.metadata or {}
    sintesis = meta.get("sintesis", "")[:200]
    print(f"  - id: {m.id} | score: {m.score:.4f} | sintesis: {sintesis}...")